# Aggregate Results of Different Types

This notebook demonstrates how to use **fan-out and fan-in patterns** in the Agent Framework to execute multiple tasks in parallel and aggregate their results, even when they return different types.

## Key Concepts

### Fan-Out Pattern
- **Fan-out** allows a single executor to dispatch work to multiple executors in parallel
- Uses `add_fan_out_edges()` to create parallel execution paths
- Each target executor receives a copy of the input data
- All parallel executors run concurrently

### Fan-In Pattern
- **Fan-in** collects results from multiple parallel executors into a single point
- Uses `add_fan_in_edges()` to aggregate results
- Results are collected as a list in the order of edge definition
- Supports heterogeneous result types using union types

### Union Type Handling
- Python's union types (`int | float`) allow different result types
- Type checking ensures type safety at compile time
- Runtime type inspection enables flexible result processing

## Workflow Architecture

```
Dispatcher (generates numbers)
    ├──> AverageExecutor (computes average → float)
    └──> SumExecutor (computes sum → int)
             ↓
         Aggregator (collects list[int | float])
```

## What This Example Shows

1. **Parallel Task Execution**: Two different computations run simultaneously
2. **Type-Safe Aggregation**: Handling mixed result types (int and float)
3. **Automatic Result Collection**: Framework manages parallel execution and collection
4. **Simple API**: `add_fan_out_edges()` and `add_fan_in_edges()` handle complexity

## Setup

Import the required modules from the Agent Framework.

In [ ]:
import random
from dataclasses import dataclass

from agent_framework.workflows import Executor, ExecutorContext, Workflow
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Define Message Types

We'll use dataclasses to represent the messages flowing through the workflow:

- **`Numbers`**: Contains a list of integers to process
- **`Result`**: Holds the final aggregated results (both average and sum)

In [ ]:
@dataclass
class Numbers:
    """Contains a list of numbers to process."""
    values: list[int]


@dataclass
class Result:
    """Contains the aggregated results from parallel computations."""
    results: list[int | float]  # Union type allows mixed int and float results

## Dispatcher Executor

The **DispatcherExecutor** generates random numbers and fans them out to multiple executors for parallel processing.

### Key Features:
- Generates 10 random integers between 1 and 100
- Returns a `Numbers` object that will be sent to all fan-out targets
- Serves as the entry point for the workflow

In [ ]:
class DispatcherExecutor(Executor[None, Numbers]):
    """Generates random numbers and dispatches them to parallel executors."""

    async def execute(self, ctx: ExecutorContext[None]) -> Numbers:
        numbers = [random.randint(1, 100) for _ in range(10)]
        print(f"Generated numbers: {numbers}")
        return Numbers(values=numbers)

## Parallel Processing Executors

These executors perform different computations on the same input data in parallel:

### AverageExecutor
- Computes the average of numbers
- Returns a **float** result
- Demonstrates floating-point computation

### SumExecutor
- Computes the sum of numbers
- Returns an **int** result
- Demonstrates integer computation

Both executors run **concurrently** and independently.

In [ ]:
class AverageExecutor(Executor[Numbers, float]):
    """Computes the average of numbers."""

    async def execute(self, ctx: ExecutorContext[Numbers]) -> float:
        numbers = ctx.get_input_data().values
        average = sum(numbers) / len(numbers)
        print(f"Average: {average}")
        return average


class SumExecutor(Executor[Numbers, int]):
    """Computes the sum of numbers."""

    async def execute(self, ctx: ExecutorContext[Numbers]) -> int:
        numbers = ctx.get_input_data().values
        total = sum(numbers)
        print(f"Sum: {total}")
        return total

## Aggregator Executor

The **AggregatorExecutor** fans in results from the parallel executors.

### Key Features:
- Receives `list[int | float]` - a union type accommodating both int and float
- Results arrive in the order edges were added (average first, then sum)
- Demonstrates type-safe handling of heterogeneous results
- Creates final `Result` object with aggregated data

In [ ]:
class AggregatorExecutor(Executor[list[int | float], Result]):
    """Aggregates results from parallel executors."""

    async def execute(self, ctx: ExecutorContext[list[int | float]]) -> Result:
        results = ctx.get_input_data()
        print(f"Aggregated results: {results}")
        print(f"Result types: {[type(r).__name__ for r in results]}")
        return Result(results=results)

## Build the Workflow

Now we'll construct the workflow graph with fan-out and fan-in edges.

### Workflow Construction Steps:

1. **Create executor instances**
2. **Add fan-out edges** from Dispatcher to Average and Sum executors
   - `add_fan_out_edges()` creates parallel execution paths
3. **Add fan-in edges** from Average and Sum executors to Aggregator
   - `add_fan_in_edges()` collects results into a list
4. **Set entry point** to Dispatcher

### Graph Visualization:
```
         Dispatcher
         /        \
    Average      Sum
         \        /
        Aggregator
```

In [ ]:
# Create executor instances
dispatcher = DispatcherExecutor()
average = AverageExecutor()
sum_executor = SumExecutor()
aggregator = AggregatorExecutor()

# Build the workflow
workflow = Workflow()

# Fan out from dispatcher to parallel executors
workflow.add_fan_out_edges(dispatcher, [average, sum_executor])

# Fan in from parallel executors to aggregator
workflow.add_fan_in_edges([average, sum_executor], aggregator)

# Set the entry point
workflow.set_entry_point(dispatcher)

print("Workflow constructed successfully!")
print("Graph structure:")
print("  Dispatcher → [Average, Sum] → Aggregator")

## Run the Workflow

Execute the workflow and observe the parallel execution and result aggregation.

### Expected Behavior:
1. Dispatcher generates 10 random numbers
2. Average and Sum executors run **in parallel**
3. Aggregator collects both results (float and int)
4. Final result contains both values in a type-safe list

### Note on Execution Order:
- Average and Sum may complete in any order (parallel execution)
- Results are always collected in edge definition order (average, sum)
- Type information is preserved throughout the pipeline

In [ ]:
# Run the workflow
result = await workflow.run()

print("\n" + "="*60)
print("WORKFLOW EXECUTION COMPLETE")
print("="*60)
print(f"Final result: {result}")
print(f"\nResult breakdown:")
print(f"  Average (float): {result.results[0]}")
print(f"  Sum (int): {result.results[1]}")

## Key Takeaways

### Fan-Out Pattern
- ✅ Use `add_fan_out_edges()` to dispatch work to multiple executors
- ✅ All target executors receive the same input data
- ✅ Executors run concurrently for better performance
- ✅ Simple API hides complex parallel execution management

### Fan-In Pattern
- ✅ Use `add_fan_in_edges()` to collect results from parallel executors
- ✅ Results are collected as a list in edge definition order
- ✅ Supports heterogeneous types with union type annotations
- ✅ Framework ensures all parallel tasks complete before fan-in

### Type Safety
- ✅ Union types (`int | float`) provide compile-time type checking
- ✅ Type annotations document expected result types
- ✅ Runtime type inspection enables flexible processing

### When to Use This Pattern
- ✅ Multiple independent computations on the same data
- ✅ Parallel processing for performance improvement
- ✅ Aggregating results from different task types
- ✅ Need type-safe handling of mixed result types

### Next Steps
- Try adding more parallel executors (e.g., min, max, median)
- Experiment with different union type combinations
- Add error handling for edge cases (empty lists, etc.)
- Explore conditional fan-out based on input data characteristics